In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector
from dotenv import load_dotenv
import re

# 환경변수 로드 (OPENAI_API_KEY 필요)
load_dotenv()

# DB 연결 설정
connection_string = "postgresql+psycopg2://주소"
collection_name = '현대해상'

# 임베딩 모델
embedding = OpenAIEmbeddings()

# 벡터스토어 초기화
vectorstore = PGVector(
    embeddings=embedding,
    collection_name=collection_name,
    connection=connection_string,
    use_jsonb=True
)

# PDF 로드
loader = PyPDFLoader("./현대해상약관.pdf")
documents = loader.load()

# 텍스트 분할기 (문단 > 문장 > 공백 > 글자 순서로 끊음)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=340,
    chunk_overlap=80,
    separators=["\n\n", "\n", ".", " ", ""]
)
split_docs = text_splitter.split_documents(documents)

# 🔧 텍스트 정리 함수
def clean_text(text: str) -> str:
    text = text.replace("\x00", "")             # 널 문자 제거
    text = re.sub(r"[\r\n]+", "\n", text)       # 줄바꿈 정리
    return text.strip()

# 정리된 문서 생성
clean_docs = []
for doc in split_docs:
    doc.page_content = clean_text(doc.page_content)
    clean_docs.append(doc)

# DB에 저장
vectorstore.add_documents(clean_docs)
print(f"총 {len(clean_docs)}개의 청크가 {collection_name} 테이블에 저장되었습니다.")